In [2]:
import numpy as np
import matplotlib.pyplot as plt

import jax
from jax import numpy as jnp, random, jit, lax

import flax
from flax import linen as nn

# init jax with some random compute. 
# JAX might complain about not having access to a GPU or TPU.
_ = jnp.square(2.)
from jax import grad, jit, vmap
from jax import random

ImportError: cannot import name 'optim' from 'flax' (/opt/conda/lib/python3.10/site-packages/flax/__init__.py)